<a href="https://colab.research.google.com/github/florianreyes/toxicity_analysis/blob/main/toxicity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
df = pd.read_csv('/content/train.csv')

In [9]:
df.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


### 1) Pre-processing the data

In [8]:
from tensorflow.keras.layers import TextVectorization

##### Separate X and y data

In [19]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [21]:
MAX_FEATURES = 20000

In [22]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length = 1800, output_mode = "int")

##### Adapt the vectorizer to our data

In [23]:
vectorizer.adapt(X.values)

##### Create the matrix with the tokenized data
##### Shape (observations x output_sequence_length)

In [27]:
vectorized_text = vectorizer(X.values)

In [28]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[ 645,   76,    2, ...,    0,    0,    0],
       [   1,   54, 2489, ...,    0,    0,    0],
       [ 425,  441,   70, ...,    0,    0,    0],
       ...,
       [   1, 7392,  383, ...,    0,    0,    0],
       [   5,   12,  534, ...,    0,    0,    0],
       [   5,    8,  130, ...,    0,    0,    0]])>

#### Create data pipeline

In [29]:
# Map - Cache - Shuffle - Batch - Prefetch

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>